<a href="https://colab.research.google.com/github/spaceluvr/face-recognition-attendance-system-/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566171 sha256=9bcfc2a613daa24a43716ec96a90024b9dff1f33f0bfd0593f81c379d060c1b1
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices("GPU"))

GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from google.colab import drive

drive.mount('/content/drive')


dataset_folder = '/content/drive/My Drive/LMS 2.0/'

Mounted at /content/drive


In [ ]:
from pathlib import Path

import face_recognition
import pickle
from IPython.display import display

DEFAULT_ENCODINGS_PATH = Path("/content/drive/MyDrive/LMS 2.0/output/encodings.pkl")

Path("training").mkdir(exist_ok=True)
Path("output").mkdir(exist_ok=True)
Path("validation").mkdir(exist_ok=True)

In [ ]:
def encode_known_faces(
        model: str = "hog", encodings_location: Path = DEFAULT_ENCODINGS_PATH
) -> None:
    names = []
    encodings = []
    for filepath in Path("/content/drive/MyDrive/LMS 2.0/training").glob("*/*"):
        if filepath.name.endswith(".DS_Store"):
            continue
        name = filepath.parent.name
        image = face_recognition.load_image_file(filepath)

        face_locations = face_recognition.face_locations(image, model=model)
        face_encodings = face_recognition.face_encodings(image, face_locations)

        for encoding in face_encodings:
            names.append(name)
            encodings.append(encoding)

        name_encodings = {"names": names, "encodings": encodings}
        with encodings_location.open(mode="wb") as f:
            pickle.dump(name_encodings, f)

In [ ]:
encode_known_faces()

In [ ]:
from pathlib import Path
from collections import Counter
from PIL import Image, ImageDraw

import face_recognition
import pickle


DEFAULT_ENCODINGS_PATH = Path("/content/drive/MyDrive/LMS 2.0/output/encodings.pkl")

Path("training").mkdir(exist_ok=True)
Path("output").mkdir(exist_ok=True)
Path("validation").mkdir(exist_ok=True)


def recognize_faces(
        input_image,
        model: str = "hog",
        encodings_location: Path = DEFAULT_ENCODINGS_PATH,
) -> None:
    with encodings_location.open(mode="rb") as f:
        loaded_encodings = pickle.load(f)


    input_face_locations = face_recognition.face_locations(
        input_image, model=model
    )

    input_face_encodings = face_recognition.face_encodings(
        input_image, input_face_locations
    )

    recognized_name = "Unknown"

    for bounding_box, unknown_encoding in zip(
            input_face_locations, input_face_encodings
    ):
        name = _recognize_face(unknown_encoding, loaded_encodings)
        if name != "Unknown":
          recognized_name = name

    return recognized_name


def _recognize_face(unknown_encoding, loaded_encodings, similarity_threshold=0.5):
    boolean_matches = face_recognition.compare_faces(
        loaded_encodings["encodings"], unknown_encoding, tolerance=similarity_threshold
    )

    # this is a list due to the model training on multiple photos, hence one  person will have multiple encodings
    recognized_names = [name for match, name in zip(boolean_matches, loaded_encodings["names"]) if match]

    if recognized_names:  # checks if the list is empty or not
        votes = Counter(recognized_names)  # returns a dictionary containig the names and the count of occurence of each
        most_common_name = votes.most_common(1)[0][0]  # most_common(1) returns a list of tuples containing the most_common values. It selects the first one and returns that
        return most_common_name
    else:
        return "Unknown"

In [ ]:
from IPython.display import display, Javascript, HTML, JSON
import base64
import io
from google.colab import output
import numpy as np


# Define the process_frame function to process each frame

def process_frames(framesData):
    names = []
    #print("Entering for loop")
    for dataURL in framesData:
        #print("Entered for loop")
        try:
            if len(dataURL) <= 6:
              continue
            #print(len(dataURL))
            image_bytes = base64.b64decode(dataURL.split(',')[1])
            #print("Step 1 done")
            pil_image = Image.open(io.BytesIO(image_bytes))
            image = np.array(pil_image)
            #print("Step 2 done")
            #print("Calling function")
            name = recognize_faces(image, model="hog")
            #print("Name Returned, appending now")
            names.append(name)
        except Exception as e:
            print("-----------------")
            print("Error processing frame:", str(e))
            print("-----------------")

    #print(names)
    counter = Counter(names)
    if counter:
      name = counter.most_common(1)[0][0]
      if name == "Unknown":
        print("Face not matched, please try again")
      else:
        print(f"Attendance Marked for {name}")
    else:
      print("Error fetching frames, please try again")


output.register_callback('process_frames', process_frames)

# JavaScript code to access the webcam and display the video feed
js_code = """
const videoElement = document.createElement('video');
document.body.appendChild(videoElement);

navigator.mediaDevices.getUserMedia({ video: true })
  .then(function(stream) {
    videoElement.srcObject = stream;
    videoElement.play();
    videoElement.style.transform = 'scaleX(-1)';
  })
  .catch(function(error) {
    console.error('Error accessing the webcam:', error);
  });

  let captureInterval;
  let framesData = [];

  function captureFrame() {
  const canvas = document.createElement('canvas');
  const context = canvas.getContext('2d');
  canvas.width = videoElement.videoWidth;
  canvas.height = videoElement.videoHeight;
  context.drawImage(videoElement, 0, 0, canvas.width, canvas.height);

  const dataURL = canvas.toDataURL('image/jpeg');
  console.log(dataURL)
  framesData.push(dataURL);
  console.log('Frame Captured');

  if (framesData.length >= 6) {
    clearInterval(captureInterval);
    google.colab.kernel.invokeFunction('process_frames', [framesData]);
    console.log('function called')
  }
}

captureInterval = setInterval(captureFrame, 1000);

"""

# Display the video feed in the Colab notebook
display(Javascript(js_code))


In [ ]:
js_code = """
const videoElement = document.createElement('video');
document.body.appendChild(videoElement);

navigator.mediaDevices.getUserMedia({ video: true })
  .then(function(stream) {
    videoElement.srcObject = stream;
    videoElement.play();
    videoElement.style.transform = 'scaleX(-1)';
  })
  .catch(function(error) {
    console.error('Error accessing the webcam:', error);
  });

  let captureInterval;
  let framesData = [];

  function captureFrame() {
  const canvas = document.createElement('canvas');
  const context = canvas.getContext('2d');
  canvas.width = videoElement.videoWidth;
  canvas.height = videoElement.videoHeight;
  context.drawImage(videoElement, 0, 0, canvas.width, canvas.height);

  const dataURL = canvas.toDataURL('image/jpeg');
  console.log(dataURL)
  framesData.push(dataURL);
  console.log('Frame Captured');

}

captureInterval = setInterval(captureFrame, 1000);

"""